In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf

from DoubleLinkedDict import bijection2int
magicDict = bijection2int()

from bench_util import *

In [2]:
#set the path and the class file
embedding_path = ".//nerTaggedwTags_150w"
class_file = "ner.classes"

In [3]:


#AutoDetect dims
with open(embedding_path, 'r' , encoding="utf-8")  as file:
    line0 = file.readline()
    dimensions = len(line0.split())-1
print("dimensions "+str(dimensions))

matrix = []
with open(embedding_path, 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        entry = line.split()
        word, values = entry[0].strip(), entry[1:]
        
        magicDict.add(word)
        vector = np.asarray(values, "double")
        matrix[magicDict.getId(word),:] = vector

#clean up
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)# only use normalised version !!!
matrix = []

dimensions 500


# Qualitative Analysis

In [4]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list

def unzip(some_list):
    return [ i for i, j in some_list ]
    
# take nearest 6 and remove the searched word itself
# print(unzip(find_nearest_k('bank' ,7)[1:]))#the food or the place
# print(unzip(find_nearest_k('apple',7)[1:]))#location or to to speak to
# print(unzip(find_nearest_k('so'   ,7)[1:]))#noun (animal) or verb 
# print(unzip(find_nearest_k('desert',   6)[1:]))#animal or verb
# print(unzip(find_nearest_k('left',   6)[1:]))#adverb (direction) and verb(the plane left)
# print(unzip(find_nearest_k('duck',   6)[1:]))# The president of the bank walked along the river bank.
# print("--------")

# Prepare Analogies.

In [5]:
def find_candidate(sim,exclusion_list_ids,isMult):
    for exclude_id in exclusion_list_ids:
        if isMult:
             sim[exclude_id] = 0#lowest possible value in 3CosMult
        else:
            sim[exclude_id] = -1#lowest possible value in 3CosAdd
    return np.argmax(sim)

In [6]:
#prepare classes of anaglogies.
#ner.classes or treebank_pos.classes or universal_pos.classes
inTags = []
outTags = []
with open(".\\datasets\\question-data\\"+class_file) as classes:
    lines = [line.split("#")[0] for line in classes.readlines()]
    for class_line in lines:
        if "=>" in class_line:
            parts = class_line.split("=>")
            inTags.append(parts[0].strip())
            outTags.append(parts[1].strip())
        else:
            inTags.append(class_line)
            outTags.append(class_line)
print(inTags)
print(outTags)

['GPE', 'GPE', 'GPE', '', '', '', '', '', '', '', 'GPE', '', '', '']
['GPE', 'GPE', 'GPE', '', '', '', '', '', '', '', 'NORP', '', '', '']


In [7]:
#step one, find all the tags.
used_tags = {}
for word in magicDict.getThings():#a little hacky, but small application
    if chr(4) in word:
        tag = word.split(chr(4))[1]
        if tag not in used_tags:
            used_tags[tag] = True
print(used_tags)

#step two, split word_dict into these tags.
dict_of_tagdicts      = {}
dict_of_Norm_matrices = {}

for tag in used_tags:
    dict_of_tagdicts[tag] = bijection2int()
    dict_of_Norm_matrices[tag] = []
    
#benchmark specific
for tagOrTags in list(set(inTags) | set(outTags)):
    tagOrTags = tagOrTags.lower()
    if "," in tagOrTags:#if multi tag
        dict_of_tagdicts[tagOrTags] = bijection2int()
        dict_of_Norm_matrices[tagOrTags] = []

for tagged_token in magicDict.getThings():
    if chr(4) in tagged_token:
        split = tagged_token.split(chr(4))
        word,tag  = split[0],split[1]
        
        id_base = magicDict.getId(tagged_token)
        dict_of_Norm_matrices[tag].append(matrix_normalized[id_base])
        
        dict_of_tagdicts[tag].add(word)
        for  key in dict_of_tagdicts:
            if "," in key and tag in key:
                key = key.lower()
                dict_of_tagdicts[key].add(word)
                dict_of_Norm_matrices[key].append(matrix_normalized[id_base])

{'loc': True, 'person': True, 'work_of_art': True, 'gpe': True, '': True, 'quantity': True, 'org': True, 'law': True, 'event': True, 'cardinal': True, 'date': True, 'norp': True, 'fac': True, 'product': True, 'ordinal': True, 'percent': True, 'time': True, 'money': True, 'language': True}


# 3cosAdd normalised befor arithmetic

In [8]:
def three_cos_predict_np_norm(a,a_star,b,b_star,dict1,dict2,m1,m2):
    if not (dict1.containsAll([a,b]) and dict2.containsAll([a_star,b_star])):
        return None
    
    id_a = dict1.getId(a)
    id_b = dict1.getId(b)
    id_a_star = dict2.getId(a_star)
    
    weight_a = m1[id_a]
    weight_b = m1[id_b]
    weight_a_star = m2[id_a_star]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    if dict1 == dict2:
        exclusion = [id_a,id_b,id_a_star]
    else:
        exclusion = [id_a_star]#the other are not part of the dict
    sim = tf.tensordot(tf.convert_to_tensor(m2),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim.numpy(),exclusion,False)
    return dict2.getThingById(index)

## 3cos Mult

In [9]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star,dict1,dict2,m1,m2):
    if not (dict1.containsAll([a,b]) and dict2.containsAll([a_star,b_star])):
        return None
    
    id_a = dict1.getId(a)
    id_b = dict1.getId(b)
    id_a_star = dict2.getId(a_star)
    
    if dict1 == dict2:
        exclusion = [id_a,id_b,id_a_star]
    else:
        exclusion = [id_a_star]
    
    weight_a = m1[id_a]
    weight_b = m1[id_b]
    weight_a_star = m2[id_a_star]
    
    nominator = sim_01(m2,weight_a_star) * sim_01(m2,weight_b) 
    denominator = sim_01(m2,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim.numpy(),exclusion,True)
    return dict2.getThingById(index)

# Test all

In [10]:
def format_results(add,mult):
    add_string = str(add)
    add_string = add_string.lstrip('0')
    add_string = format(add_string, ".4")
    mult_string = str(mult)
    mult_string = mult_string.lstrip('0')
    mult_string = format(mult_string, ".4")
    return add_string +'/'+mult_string


In [11]:
def find_variants(word,magicGlobal):
    ids= []
    for w in magicGlobal.getThings():
        if w.startswith(word+chr(4)):
            ids.append(magicGlobal.getId(w))
    return ids

print(find_variants("kuna",magicDict))
print(magicDict.getThingById(106965))
print(magicDict.getThingById(147466))

[76157, 163261, 164074]
nodorg
bachata


In [12]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0,0]
count_syn_sucess      = [0,0,0]

errors = []

files = os.listdir('.\\datasets\\question-data')
for idx,file_name in enumerate(sorted(files)):
    print(file_name)
    with open('.\\datasets\\question-data\\'+file_name,'r') as file:
        if file_name.endswith('.classes'):
            continue

        outOfVocab = 0
        
        inTag = inTags[idx].lower()
        outTag = outTags[idx].lower()
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
        
        dual_class_in  = False
        dual_class_out = False
        
        
        dict1 = dict_of_tagdicts[inTag]
        m1    = dict_of_Norm_matrices[inTag]
        
        dict2 = dict_of_tagdicts[outTag]
        m2    = dict_of_Norm_matrices[outTag]
        
        
        for line_id,line in enumerate(lines):
            if (line_id % 100 == 0):
                print(line_id)
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            
            
            
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star,dict1,dict2,m1,m2)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star,dict1,dict2,m1,m2)
            #clean up, of tagged predictions

            if predicted_3cos_add_norm == None:
                outOfVocab+= 1


            if predicted_3cos_add_norm != None and chr(4) in predicted_3cos_add_norm:
                predicted_3cos_add_norm = predicted_3cos_add_norm.split(chr(4))[0]
            if predicted_3cos_mult != None and chr(4) in predicted_3cos_mult:
                predicted_3cos_mult = predicted_3cos_mult.split(chr(4))[0]
            
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_sem_sucess[2] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_syn_sucess[2] += 1
                    
        print(format_results(local_sucesses[1]/float(local_tasks_count),local_sucesses[2]/float(local_tasks_count)))
        print("out of Vocab:"+str(outOfVocab))
        print('-------------------')

print('semantical')
print(format_results(count_sem_sucess[1]/float(count_sem_questions),count_sem_sucess[2]/float(count_sem_questions)))

print('syntactical')
print(format_results(count_syn_sucess[1]/float(count_syn_questions),count_syn_sucess[2]/float(count_syn_questions)))

print('overall')
print(format_results((count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_syn_questions), \
                     (count_syn_sucess[2]+count_sem_sucess[2])/float(count_sem_questions+count_syn_questions)))

capital-common-countries.txt
0
100
200
300
400
500
.685/.618
out of Vocab:0
-------------------
capital-world.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
.450/.379
out of Vocab:308
-------------------
city-in-state.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
.512/.400
out of Vocab:73
-------------------
currency.txt
0
100
200
300
400
500
600
700
800
.017/.020
out of Vocab:168
-------------------
family.txt
0
100
200
300
400
500
.658/.602
out of Vocab:0
-------------------
gram1-adjective-to-adverb.txt
0
100
200
300
400
500
600
700
800
900
.141/.164
out of Vocab:0
-------------------
gram2-opposite.txt
0
100
200
300
400
500
600
700
800
.259/.251
out of Vocab:0
-------------------
gram3-comparative.txt
0
100
200
300
400
500
600
700
800
900
1

In [13]:
import winsound
winsound.Beep(440, 500)

# Spearman Rank

In [14]:
with open(r'.\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            print(id1)
            print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = sim_01(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))

linetiger	cat	7.35

tiger cat
0 0.735
linetiger	tiger	10.00

tiger tiger
0 1.0
lineplane	car	5.77

plane car
0 0.577
linetrain	car	6.31

train car
0 0.631
linetelevision	radio	6.77

television radio
0 0.6769999999999999
linemedia	radio	7.42

media radio
0 0.742
linebread	butter	6.19

bread butter
0 0.619
linecucumber	potato	5.92

cucumber potato
0 0.592
linedoctor	nurse	7.00

doctor nurse
0 0.7
lineprofessor	doctor	6.62

professor doctor
0 0.662
linestudent	professor	6.81

student professor
0 0.6809999999999999
linesmart	stupid	5.81

smart stupid
0 0.581
linewood	forest	7.73

wood forest
0 0.773
linemoney	cash	9.15

money cash
0 0.915
lineking	queen	8.58

king queen
0 0.858
lineking	rook	5.92

king rook
0 0.592
linebishop	rabbi	6.69

bishop rabbi
0 0.669
linefuck	sex	9.44

fuck sex
0 0.944
linefootball	soccer	9.03

football soccer
0 0.9029999999999999
linefootball	basketball	6.81

football basketball
0 0.6809999999999999
linefootball	tennis	6.63

football tennis
0 0.663
lineArafat	Jack

In [15]:
from scipy import stats
stats.spearmanr(losses, scores)

C:\Users\weso\anaconda3\envs\gaundlet-TF\lib\site-packages\scipy\stats\stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


SpearmanrResult(correlation=nan, pvalue=nan)